<a href="https://colab.research.google.com/github/filippozuddas/ML-SRT-SETI/blob/main/notebooks/train_CVAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import os

# --- CONFIGURAZIONE PERCORSI ---
# Il percorso che hai appena trovato
PROJECT_ROOT = "/content/filippo/ML-SRT-SETI"

# 1. Verifica di sicurezza
if not os.path.exists(PROJECT_ROOT):
    raise FileNotFoundError(f"Attenzione! Non trovo la cartella: {PROJECT_ROOT}")

# 2. Spostiamo la 'Working Directory' lì
# Così quando fai np.load("data/processed/...") lui parte dalla root corretta
os.chdir(PROJECT_ROOT)

# 3. Aggiungiamo la cartella a Python per gli import
# Questo permette di fare 'from src import models' anche se il notebook non è lì fisicamente
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)

print(f"✅ Setup Completato!")
print(f"   Directory di lavoro: {os.getcwd()}")

✅ Setup Completato!
   Directory di lavoro: /content/filippo/ML-SRT-SETI


In [2]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Setup path per importare src
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from src import config, models

# Verifica GPU
print(f"TensorFlow Version: {tf.__version__}")
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

TensorFlow Version: 2.18.0
Num GPUs Available:  2


In [3]:
# Percorso del file generato nello step precedente
DATA_PATH = "data/processed/train_class0_5k.npy"

# 1. Carica i dati
print(f"Caricamento dati da {DATA_PATH}...")
dataset_cadences = np.load(DATA_PATH)
print(f"Shape originale (Cadenze): {dataset_cadences.shape}")

# 2. Reshape: Da (N_samples, 6, 16, 512, 1) a (N_samples * 6, 16, 512, 1)
# Fondiamo la dimensione delle cadenze con quella dei campioni
# -1 dice a numpy di calcolare automaticamente la dimensione
X_all = dataset_cadences.reshape(-1, config.TCHANS, config.FCHANS_FINAL, 1)

print(f"Shape per il training (Frames singoli): {X_all.shape}")

# 3. Train/Test Split
# Usiamo il random_state per riproducibilità
X_train, X_val = train_test_split(X_all, test_size=0.2, random_state=42)

print(f"Training frames: {X_train.shape}")
print(f"Validation frames: {X_val.shape}")

Caricamento dati da data/processed/train_class0_5k.npy...
Shape originale (Cadenze): (5000, 6, 16, 512, 1)
Shape per il training (Frames singoli): (30000, 16, 512, 1)
Training frames: (24000, 16, 512, 1)
Validation frames: (6000, 16, 512, 1)


In [4]:
# Parametri
input_shape = (config.TCHANS, config.FCHANS_FINAL, 1)
latent_dim = config.LATENT_DIM # 8

print(f"Costruzione modello con input {input_shape} e latent dim {latent_dim}...")

# 1. Encoder & Decoder
encoder = models.build_encoder(input_shape, latent_dim)
decoder = models.build_decoder(latent_dim, target_shape=input_shape)

# 2. CVAE Completo
# beta=1.0 è un VAE standard.
# beta > 1 forza una rappresentazione latente più "districata" (disentangled)
cvae = models.CVAE(encoder, decoder, beta=1.0)

# Compile
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
cvae.compile(optimizer=optimizer)

# (Opzionale) Visualizza sommario encoder
encoder.summary()

Costruzione modello con input (16, 512, 1) e latent dim 8...


Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 16, 512,   │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 8, 256,    │        320 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 8, 256,    │        128 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 4, 128,    │     18,496 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 4, 128,    │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 2, 64,     │     73,856 │ batch_normalizat… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 2, 64,     │        512 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 16384)     │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │  2,097,280 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ z_mean (Dense)      │ (None, 8)         │      1,032 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ z_log_var (Dense)   │ (None, 8)         │      1,032 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sampling (Sampling) │ (None, 8)         │          0 │ z_mean[0][0],     │
│                     │                   │            │ z_log_var[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,192,912 (8.37 MB)

 Trainable params: 2,192,464 (8.36 MB)

 Non-trainable params: 448 (1.75 KB)

In [5]:
# Configurazione Training
EPOCHS = 20
BATCH_SIZE = config.BATCH_SIZE # 32

print("Avvio training...")

history = cvae.fit(
    X_train,
    validation_data=(X_val, X_val), # Nel VAE input e target sono uguali (ricostruzione)
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    shuffle=True
)

print("Training completato.")

Avvio training...
Epoch 1/20
742/750 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - kl_loss: 36.6378 - loss: 8720.3965 - reconstruction_loss: 8683.7578

ValueError: No loss to compute. Provide a `loss` argument in `compile()`.